# MODEL EXPORT
- Compact Model : write final graph (in pb format) 재학습 불가
- Servabble Model : Converts a checkpoint and graph to a servable for TensorFlow Serving. (Tensorpack 의존성 없이 Serving에서 사용가능)

In [ ]:
# Export 
# Package Load AND PATH Setting

import argparse
import logging
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"]="99"
sys.path.append(os.path.abspath('./src'))

from tensorpack import *
from tensorpack.predict import PredictConfig
from tensorpack.tfutils import SmartInit
from tensorpack.tfutils.export import ModelExporter

from config import config as cfg
from config import finalize_configs
from dataset import register_custom_data
from modeling.generalized_rcnn import ResNetC4Model, ResNetFPNModel

from tensorpack.utils import logger

def check_and_make_dir(path):
    dir_path = os.path.dirname(path)
    if not os.path.exists(dir_path):
        logger.info("Create new dir '{}'".format(dir_path))
        os.makedirs(dir_path)


EXPORT_CONFIG_PATH = './configs/default_config.json'
LOAD_MODEL_PATH = './train_log/default_config_model/export/max-eval-mAP'

SERVABLE_VERSION = 1
EXPORT_SERVABLE_PATH = './train_log/default_config_model/servable_model/'+ str(SERVABLE_VERSION)
EXPORT_COMPACT_MODEL = './train_log/default_config_model/compact_model/compact_model.pb'

In [ ]:
# update config

cfg.update_from_json(EXPORT_CONFIG_PATH)
cfg.DATA.CLASS_NAMES = ['BG'] + cfg.DATA.CLASS_NAMES
    
register_custom_data()
finalize_configs(is_training=False)

# initialize model
model = ResNetFPNModel() if cfg.MODE_FPN else ResNetC4Model()


logger.info("Loading checkpoint from {}".format(LOAD_MODEL_PATH))
predcfg = PredictConfig(model=model,
                        session_init=SmartInit(LOAD_MODEL_PATH),
                        input_names=model.get_inference_tensor_names()[0],
                        output_names=model.get_inference_tensor_names()[1])

# compact 모델 export
check_and_make_dir(EXPORT_COMPACT_MODEL)
ModelExporter(predcfg).export_compact(EXPORT_COMPACT_MODEL,optimize=True)
    
# servablbe 모델 export
check_and_make_dir(EXPORT_SERVABLE_PATH)
ModelExporter(predcfg).export_serving(EXPORT_SERVABLE_PATH)